In [1]:
!pip install transformers accelerate bert-score --quiet
!pip install timeout-decorator --quiet
!pip uninstall -y pylibcudagraph-cu12 rmm-cu12 --quiet
!pip install transformers==4.40.1 peft==0.10.0 --quiet
!pip install scikit-learn --quiet
!pip install timeout-decorator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.4 MB/s eta 0:00:00
ERROR

In [2]:
!pip freeze > requirements.txt

In [3]:
import os
import gc
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForQuestionAnswering
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from timeout_decorator import timeout, TimeoutError
from bert_score import score

In [4]:
# Configurable parameter
MAX_NEW_TOKENS = 32  # Maximum number of new tokens for model generation

# Load your curated dataset
base_path = "/kaggle/input/vr-dataset/dataset_curated"
subfolders = [f"S{i}" for i in range(1, 7)]

In [5]:
all_dfs = []

for folder in subfolders:
    csv_path = os.path.join(base_path, folder, f"{folder}_qa_data.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, header=None, names=["image_path", "question", "answer"])
        df["image_path"] = df["image_path"].apply(
            lambda p: os.path.join(base_path, os.path.normpath(p).split("dataset_curated/")[-1])
        )
        df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)
        all_dfs.append(df)
        print(f"Loaded {len(df)} examples from {csv_path}")
    else:
        print(f"CSV not found: {csv_path}")

df_all = pd.concat(all_dfs).reset_index(drop=True)

# Clean the dataset: Remove rows with NaN or empty questions and answers
df_all = df_all[df_all["question"].notna() & df_all["answer"].notna()]
df_all["question"] = df_all["question"].astype(str)
df_all["answer"] = df_all["answer"].astype(str)
df_all = df_all[df_all["question"].str.strip() != ""]
df_all = df_all[df_all["answer"].str.strip() != ""]
print(f"After cleaning, total QA pairs: {len(df_all)}")

# Select the first 5000
df_subset = df_all.iloc[:5000]

Loaded 14366 examples from /kaggle/input/vr-dataset/dataset_curated/S1/S1_qa_data.csv
Loaded 14358 examples from /kaggle/input/vr-dataset/dataset_curated/S2/S2_qa_data.csv
Loaded 14367 examples from /kaggle/input/vr-dataset/dataset_curated/S3/S3_qa_data.csv
Loaded 14366 examples from /kaggle/input/vr-dataset/dataset_curated/S4/S4_qa_data.csv
Loaded 14387 examples from /kaggle/input/vr-dataset/dataset_curated/S5/S5_qa_data.csv
Loaded 14376 examples from /kaggle/input/vr-dataset/dataset_curated/S6/S6_qa_data.csv
After cleaning, total QA pairs: 86208


In [6]:
# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("owais-md/blip-vqa-r5-e7")
model = BlipForQuestionAnswering.from_pretrained("owais-md/blip-vqa-r5-e7")
device = torch.device("cuda")
model.to(device)
model.eval()

2025-05-18 16:37:38.518784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747586258.693449      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747586258.743223      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.98M [00:00<?, ?B/s]

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [7]:
# Timeout-wrapped function to predict answer
@timeout(10)
def predict_answer(image_path, question):
    try:
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            return ""
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, text=question, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)
        predicted_answer = processor.decode(outputs[0], skip_special_tokens=True).strip().lower()
        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()
        return predicted_answer
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

In [8]:
# No slicing, just use the full df_subset directly
ground_truths = df_subset["answer"].tolist()
predictions = []
skipped_entries = []  # To track any timeouts if they happen

# Loop over the full dataframe with a progress bar
for idx, row in tqdm(df_subset.iterrows(), total=len(df_subset), desc="Processing"):
    # Get the image path and question from the row
    full_image_path = row["image_path"]
    question = row["question"]
    
    # Try to predict the answer
    try:
        predicted = predict_answer(full_image_path, question)
    except TimeoutError:
        print(f"Timeout processing row {idx}: {full_image_path}")
        skipped_entries.append({"row": idx, "image_path": full_image_path, "question": question})
        predicted = ""  # Default to empty string for timeouts
    except Exception as e:
        print(f"Other error processing row {idx}: {e}")
        predicted = ""  # Default to empty string for other errors
    
    # Append the prediction
    predictions.append(predicted)
    
    # Quick check to make sure predictions are keeping up
    if len(predictions) != idx + 1:
        print(f"Warning: predictions length mismatch at idx {idx}")
    
    # Compute metrics every 1000 samples or at the end
    if (idx + 1) % 1000 == 0 or (idx + 1) == len(df_subset):
        y_true_temp = [str(g).lower() for g in ground_truths[:idx+1]]
        y_pred_temp = [str(p).lower() for p in predictions[:idx+1]]
        
        if len(y_true_temp) != len(y_pred_temp):
            print(f"Length mismatch at idx {idx}: y_true_temp={len(y_true_temp)}, y_pred_temp={len(y_pred_temp)}")
        else:
            accuracy = accuracy_score(y_true_temp, y_pred_temp)
            f1 = f1_score(y_true_temp, y_pred_temp, average='macro', zero_division=0)
            print(f"After {idx + 1} samples: Accuracy = {accuracy:.4f}, F1 Score = {f1:.4f}")

# Final check for all predictions
if len(predictions) != len(df_subset):
    print(f"Final mismatch: expected {len(df_subset)}, got {len(predictions)}")
else:
    # Calculate final metrics
    y_true = [str(g).lower() for g in ground_truths]
    y_pred = [str(p).lower() for p in predictions]
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    print(f"Final Accuracy: {accuracy:.4f}")
    print(f"Final F1 Score: {f1:.4f}")

# Let you know if any rows were skipped due to timeouts
if skipped_entries:
    print(f"Skipped {len(skipped_entries)} entries due to timeouts")

Processing:  20%|██        | 1000/5000 [07:29<30:02,  2.22it/s]

After 1000 samples: Accuracy = 0.6970, F1 Score = 0.4083


Processing:  40%|████      | 2000/5000 [15:04<23:59,  2.08it/s]

After 2000 samples: Accuracy = 0.7000, F1 Score = 0.3832


Processing:  60%|██████    | 3000/5000 [22:58<15:50,  2.10it/s]

After 3000 samples: Accuracy = 0.7000, F1 Score = 0.3587


Processing:  80%|████████  | 4000/5000 [30:52<08:05,  2.06it/s]

After 4000 samples: Accuracy = 0.6997, F1 Score = 0.3510


Processing: 100%|██████████| 5000/5000 [38:46<00:00,  2.15it/s]

After 5000 samples: Accuracy = 0.6926, F1 Score = 0.3294
Final Accuracy: 0.6926
Final F1 Score: 0.3294


In [9]:
# Final evaluation
y_true = [str(g).lower() for g in ground_truths]
y_pred = [str(p).lower() for p in predictions]
correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
total = len(y_true)
accuracy = (correct / total) * 100 if total > 0 else 0
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)

# Compute BERTScore
print("Computing BERTScore...")
P, R, F1 = score(y_pred, y_true, lang="en", verbose=True)
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

# Print final results
print(f"\nFinal Results:")
print(f"Total Questions: {total}")
print(f"Correct Predictions: {correct}")
print(f"Exact Match Accuracy: {accuracy:.2f}%")
print(f"Exact Match Macro Precision: {precision:.2f}")
print(f"Exact Match Macro Recall: {recall:.2f}")
print(f"Exact Match Macro F1 Score: {f1:.2f}")
print(f"BERTScore Precision: {bertscore_precision:.4f}")
print(f"BERTScore Recall: {bertscore_recall:.4f}")
print(f"BERTScore F1: {bertscore_f1:.4f}")
print(f"Skipped Entries: {len(skipped_entries)}")
if skipped_entries:
    skipped_df = pd.DataFrame(skipped_entries)
    print("\nSkipped Entries (due to timeout):")
    print(skipped_df)

# Display sample predictions
results_df = pd.DataFrame({
    "Image Path": df_subset["image_path"],
    "Question": df_subset["question"],
    "Ground Truth": ground_truths,
    "Predicted": predictions
})
print("\nSample Predictions (First 10):")
print(results_df.head(10))

Computing BERTScore...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/15 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/79 [00:00<?, ?it/s]

done in 1.40 seconds, 3574.99 sentences/sec

Final Results:
Total Questions: 5000
Correct Predictions: 3463
Exact Match Accuracy: 69.26%
Exact Match Macro Precision: 0.34
Exact Match Macro Recall: 0.35
Exact Match Macro F1 Score: 0.33
BERTScore Precision: 0.9864
BERTScore Recall: 0.9833
BERTScore F1: 0.9846
Skipped Entries: 0

Sample Predictions (First 10):
                                          Image Path  \
0  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
1  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
2  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
3  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
4  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
5  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
6  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
7  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
8  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   
9  /kaggle/input/vr-dataset/dataset_curated/S1/S1...   

                       